In [3]:
# I run the clustering on the gpu, maybe an environment with these dependencies will take some time to set up. Feel free to ask help.
# import cugraph
# import cudf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# import faiss
# import anndata as ad

# My own imports
import aux_funcs_v0 as f
import seaborn as sns  # Import Seaborn
import pandas as pd
import time
import warnings
import gc
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pickle
import importlib

# Suppress FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

# All columns will be displayed in pandas dataframe
pd.set_option('display.max_columns', None)

# Print version of cugraph used
# cugraph.__version__

# Hyperparameters
K_NN = 100

In [4]:
importlib.reload(f)

<module 'aux_funcs_v0' from 'd:\\00-master\\02 - Semester2\\[R1] Research_internship_1\\01-Code\\data_preparation\\aux_funcs_v0.py'>

In [6]:
emerson = pd.read_csv('../../02-Data/data_preparation/processed_files/metadata_merged_full.tsv', sep='\t')
# display(emerson[:2])

emerson_sample_name = emerson["sample_name"]

# For debugging purpouses
emerson_sample_name_reduced = emerson_sample_name[:10]
# display(emerson_sample_name[:2])

# Decide which list to use
list_files = emerson_sample_name

In [12]:
sizes = {}
for name in emerson_sample_name[:3]:
    chunks = pd.read_csv(f'../../02-Data/data_preparation/processed_files/{name}_clonotypes.tsv', sep='\t') 
    print(chunks.count())
    sizes[name] = chunks.count()

sizes


Unnamed: 0            230938
seq_reads             230938
v_resolved            230481
j_resolved            230938
cdr3_rearrangement    187003
cdr3_amino_acid       187003
dtype: int64
Unnamed: 0            279329
seq_reads             279329
v_resolved            278905
j_resolved            279329
cdr3_rearrangement    231855
cdr3_amino_acid       231855
dtype: int64
Unnamed: 0            270012
seq_reads             270012
v_resolved            269780
j_resolved            270012
cdr3_rearrangement    227356
cdr3_amino_acid       227356
dtype: int64


{'P00030': Unnamed: 0            230938
 seq_reads             230938
 v_resolved            230481
 j_resolved            230938
 cdr3_rearrangement    187003
 cdr3_amino_acid       187003
 dtype: int64,
 'P00061': Unnamed: 0            279329
 seq_reads             279329
 v_resolved            278905
 j_resolved            279329
 cdr3_rearrangement    231855
 cdr3_amino_acid       231855
 dtype: int64,
 'P00127': Unnamed: 0            270012
 seq_reads             270012
 v_resolved            269780
 j_resolved            270012
 cdr3_rearrangement    227356
 cdr3_amino_acid       227356
 dtype: int64}

1. Read emerson_full row by row and extract "sample_name"
2. Read sample_name_clonotypes.tsv file
3. Save TCR and amount of times seen in a list


Notes:
1. I am running this on 24 GB of RAM, and it is pretty intensive. Further modifications might be needed

In [7]:
# Get a List containing all sequences that appear more than once
tcr_list = f.count_list(list_files)
display(tcr_list)

# Create a dataframe from said list and save it to memory
tcr_df = pd.DataFrame(tcr_list, columns=['cdr3','count'])
# tcr_df.to_csv('data/tcr_df.csv',index=False)

Processing P00030 completed in 1.23 seconds. Progress: 0.15%
Processing P00061 completed in 0.81 seconds. Progress: 0.30%
Processing P00127 completed in 1.42 seconds. Progress: 0.45%
Processing P00033 completed in 0.77 seconds. Progress: 0.60%
Processing P00175 completed in 2.02 seconds. Progress: 0.75%
Processing P00078 completed in 0.59 seconds. Progress: 0.90%
Processing P00173 completed in 0.70 seconds. Progress: 1.05%
Processing P00069 completed in 0.99 seconds. Progress: 1.20%
Processing P00093 completed in 1.60 seconds. Progress: 1.35%
Processing P00092 completed in 1.09 seconds. Progress: 1.50%
Processing P00121 completed in 0.47 seconds. Progress: 1.65%
Processing P00024 completed in 0.60 seconds. Progress: 1.80%
Processing P00042 completed in 0.78 seconds. Progress: 1.95%
Processing P00167 completed in 1.04 seconds. Progress: 2.10%
Processing P00160 completed in 1.65 seconds. Progress: 2.25%
Processing P00161 completed in 0.64 seconds. Progress: 2.40%
Processing P00040 comple

[['CASSLEGTGNTEAFF', 136],
 ['CASSQGPTSNEQFF', 8],
 ['CASSLGETQYF', 665],
 ['CASSGLSQGSYGYTF', 2],
 ['CASSLPGQDYEQYF', 58],
 ['CASSLGYEQYF', 659],
 ['CASSYRGPSYEQYF', 111],
 ['CASSLGGYEQYF', 650],
 ['CASSPGLAGGAEQYF', 14],
 ['CASSPSSYEQYF', 644],
 ['CASSSSYEQYF', 659],
 ['CASSLGGNTEAFF', 662],
 ['CASSSQETQYF', 660],
 ['CASSLGGGTDTQYF', 626],
 ['CASSLSYEQYF', 655],
 ['CASSLSGSSYNEQFF', 584],
 ['CASSLGTEAFF', 650],
 ['CASSPGTGFVSEQYF', 2],
 ['CASSRSYEQYF', 605],
 ['CASSLVAGGPNEQFF', 204],
 ['CASSLGQGAYEQYF', 633],
 ['CASSRGLAGVREQFF', 11],
 ['CSVPAFF', 10],
 ['CASSLTDTQYF', 652],
 ['CASSSSTDTQYF', 638],
 ['CASSFTDTQYF', 648],
 ['CASRGPTGGFYGYTF', 1],
 ['CASSRGYEQYF', 610],
 ['CASSLENHPLYEQYF', 1],
 ['CASSLSTDTQYF', 658],
 ['CASSRTGGYEQYF', 511],
 ['CASSQSYEQYF', 607],
 ['CASSLADTQYF', 649],
 ['CASSSGANVLTF', 587],
 ['CASSLGGTEAFF', 655],
 ['CASSFSGANVLTF', 590],
 ['CASSLGGSYNEQFF', 608],
 ['CASSLGGSYEQYF', 648],
 ['CASRGPGKSQETQYF', 1],
 ['CASSLQGSYEQYF', 574],
 ['CASSLGDTQYF', 651],
 ['

In [3]:
# Load back in the sequences
tcr_df = pd.read_csv('data/tcr_df.csv')
tcr_df

,cdr3,count
0,CASSLEGTGNTEAFF,136
1,CASSQGPTSNEQFF,8
2,CASSLGETQYF,665
3,CASSGLSQGSYGYTF,2
4,CASSLPGQDYEQYF,58
...,...,...
66705067,CATHDPQVINGYTF,1
66705068,CASSAPTGARGYTF,1
66705069,CASSLLGQYSGETQYF,1
66705070,CASSQKDSSNTGELFF,1


In [4]:
# Define a threshold and filter some data
threshold = 10 # Some 1's got mixed in for some reason

# Clean up some sequences. Start with C - end with F,W . no * _
tcr_df = tcr_df[tcr_df['cdr3'].str.contains('^C[^*_]*[FW]$')]

tcr_df = tcr_df[tcr_df["count"]>threshold]
display(tcr_df[:8])

f.general_info(tcr_df)

,cdr3,count
0,CASSLEGTGNTEAFF,136
2,CASSLGETQYF,665
4,CASSLPGQDYEQYF,58
5,CASSLGYEQYF,659
6,CASSYRGPSYEQYF,111
7,CASSLGGYEQYF,650
8,CASSPGLAGGAEQYF,14
9,CASSPSSYEQYF,644


General Information about tcr_df:
Size of the DataFrame: 2147924, number of columns = 2, number of rows = 1073962
Maximum value in column 'count': 665, Minimum value in column 'count': 11
Number of unique occurrences: 1073962


In [16]:
importlib.reload(f)
# We want a list of where every sequence is found [seq1, file_id] ! This same sequence can appear multiple file_ids
tcr_list_samples = f.obtain_sample_name_opt(tcr_df,list_files)
tcr_list_samples

Processing P00030 completed in 0.83 seconds. Progress: 0.15%
Processing P00061 completed in 1.01 seconds. Progress: 0.30%
Processing P00127 completed in 0.83 seconds. Progress: 0.45%
Processing P00033 completed in 1.00 seconds. Progress: 0.60%
Processing P00175 completed in 1.24 seconds. Progress: 0.75%
Processing P00078 completed in 0.92 seconds. Progress: 0.90%
Processing P00173 completed in 1.10 seconds. Progress: 1.05%
Processing P00069 completed in 1.12 seconds. Progress: 1.20%
Processing P00093 completed in 1.51 seconds. Progress: 1.35%
Processing P00092 completed in 1.11 seconds. Progress: 1.50%
Processing P00121 completed in 1.40 seconds. Progress: 1.65%
Processing P00024 completed in 0.84 seconds. Progress: 1.80%
Processing P00042 completed in 1.58 seconds. Progress: 1.95%
Processing P00167 completed in 4.53 seconds. Progress: 2.10%
Processing P00160 completed in 3.35 seconds. Progress: 2.25%
Processing P00161 completed in 1.84 seconds. Progress: 2.40%
Processing P00040 comple

[        cdr3_amino_acid file_id
 3        CASSLGSNQETQYF  P00030
 5         CASSLGGPWEQYF  P00030
 6         CASSLQGSYEQYF  P00030
 8       CASSSGGARSYEQYF  P00030
 9         CASSLNTVYEQYF  P00030
 ...                 ...     ...
 230924     CASSYKNTEAFF  P00030
 230928  CASSPGDRVDEKLFF  P00030
 230934    CASSLPPYTEAFF  P00030
 230935    CASRTTVNTEAFF  P00030
 230937      CASSTNTEAFF  P00030
 
 [56940 rows x 2 columns],
         cdr3_amino_acid file_id
 1       CASSLDRSYNSPLHF  P00061
 2       CASSLSGHSSYEQYF  P00061
 6        CASSRGLETYEQYF  P00061
 8          CASSRRDSPLHF  P00061
 17      CASSLGGFSGNTIYF  P00061
 ...                 ...     ...
 279311    CASRRAGAGELFF  P00061
 279313    CATSDPDRGGYTF  P00061
 279315    CASSLAGLREAFF  P00061
 279324   CASSRRDNSYEQYF  P00061
 279325     CASSGRDQPQHF  P00061
 
 [74447 rows x 2 columns],
         cdr3_amino_acid file_id
 2       CASSLLGGNNQPQHF  P00127
 5        CASSSGHPSYEQYF  P00127
 9       CASSLGLGIQETQYF  P00127
 12        CASSPPG

In [10]:
sample_id = "P00030"
f.print_example(sample_id,tcr_df)

original file P00030_clonotypes.tsv


,Unnamed: 0,seq_reads,v_resolved,j_resolved,cdr3_rearrangement,cdr3_amino_acid
0,0,4,TCRBV05-05*01,TCRBJ02-05*01,TGTGCCAGCAGCTCCCCGAGTTCCTCCGGTTGGGGGGCGCAGGGGA...,CASSSPSSSGWGAQGTQYF
1,1,4,TCRBV05-05*01,TCRBJ02-05*01,NaN,NaN
2,2,4,TCRBV05-05*01,TCRBJ02-05*01,TGTGCCAGCAGCTCATCCACATTCGTAGGCCAAGAGACCCAGTACTTC,CASSSSTFVGQETQYF
3,3,4,TCRBV05-05*01,TCRBJ02-05*01,TGTGCCAGCAGCTTGGGGTCCAACCAAGAGACCCAGTACTTC,CASSLGSNQETQYF
4,4,6,TCRBV05-05*01,TCRBJ02-07*01,TGTGCCAGCAGCCTCCCCAAAGCCAGGGGCCGGGAGCAGTACTTC,CASSLPKARGREQYF


tcrs that repeat more than threshold value


0    CASSLEGTGNTEAFF
2        CASSLGETQYF
4     CASSLPGQDYEQYF
5        CASSLGYEQYF
6     CASSYRGPSYEQYF
Name: cdr3, dtype: object

matiching values inbetween the list and original data


,Unnamed: 0,seq_reads,v_resolved,j_resolved,cdr3_rearrangement,cdr3_amino_acid
3,3,4,TCRBV05-05*01,TCRBJ02-05*01,TGTGCCAGCAGCTTGGGGTCCAACCAAGAGACCCAGTACTTC,CASSLGSNQETQYF
5,5,6,TCRBV05-05*01,TCRBJ02-07*01,TGTGCCAGCAGCTTGGGGGGTCCGTGGGAGCAGTACTTC,CASSLGGPWEQYF
6,6,4,TCRBV05-05*01,TCRBJ02-05*01,TGTGCCAGCAGCTTGCAGGGCTCCTACGAGCAGTACTTC,CASSLQGSYEQYF
8,8,4,TCRBV05-05*01,TCRBJ02-05*01,TGTGCCAGCAGCTCTGGCGGGGCACGCTCCTACGAGCAGTACTTC,CASSSGGARSYEQYF
9,9,4,TCRBV05-05*01,TCRBJ02-05*01,TGTGCCAGCAGCTTGAATACGGTCTACGAGCAGTACTTC,CASSLNTVYEQYF


join operation on both


,cdr3,P00030
0,CASSLGSNQETQYF,True
1,CASSLGGPWEQYF,True
2,CASSLQGSYEQYF,True
3,CASSSGGARSYEQYF,True
4,CASSLNTVYEQYF,True


In [ ]:
# Now we join all cdr3 sequences 
cdr3_files = pd.concat(tcr_list_samples, ignore_index=True)

# Group by 'cdr3' column and aggregate 'sample_id' column
cdr3_files = cdr3_files.groupby('cdr3_amino_acid')['file_id'].apply(lambda x: ','.join(x)).reset_index()

cdr3_files.to_csv('data/cdr3_files.csv',index=False)
cdr3_files

In [8]:
cdr3_files = pd.read_csv('data/cdr3_files.csv')
cdr3_files

,cdr3_amino_acid,file_id
0,CAAAGGSYNEQFF,"P00079,P00023,P00457,P00651,P00467,P00623,P006..."
1,CAAAGNTEAFF,"P00194,P00194,P00003,P00031,P00111,P00463,P005..."
2,CAAASGNTIYF,"P00057,P00003,P00118,P00467,P00660,P00418,P006..."
3,CAADSGNTIYF,"P00019,P00017,P00011,P00053,P00504,P00510,P006..."
4,CAADTQYF,"P00121,P00089,P00019,P00197,P00009,P00136,P006..."
...,...,...
1073957,CVGGYTF,"P00027,P00089,P00194,P00128,P00168,P00564,P005..."
1073958,CVIKQKTAYEQYF,"P00042,P00083,P00138,P00182,P00176,P00062,P001..."
1073959,CVKQKTAYEQYF,"P00160,P00090,P00186,P00019,P00019,P00097,P001..."
1073960,CVSKQKPAYEQYF,"P00141,P00141,P00141,P00141,P00025,P00147,P001..."


In [ ]:
# Split file_id into individual IDs and create columns
cdr3_files['file_id'] = cdr3_files['file_id'].apply(lambda x: x.split(','))
file_ids = sorted(set([id for sublist in cdr3_files['file_id'] for id in sublist]))
for file_id in file_ids:
    cdr3_files[file_id] = cdr3_files['file_id'].apply(lambda x: file_id in x)

# Rearrange columns
cdr3_files = cdr3_files[['cdr3_amino_acid'] + file_ids]

cdr3_files

In [11]:
cdr3_files.to_csv('data/cdr3_files_f2.csv',index=False)


In [3]:
metadata = pd.read_csv('data/processed_files/metadata_merged_full.tsv', sep='\t')
cdr3_files = pd.read_csv('data/cdr3_files.csv')

# Select columns 2 to 10 from the metadata dataframe
metadata_columns = metadata.iloc[:, 18:288]

# Create a new DataFrame by combining the selected columns
hla_info = pd.concat([cdr3_files.iloc[:, 0], metadata_columns], axis=1)

# Fill the DataFrame with False values
hla_info = hla_info.fillna(False, downcast='infer')

hla_info.to_csv('data/hla_info.csv',index=False)

hla_info

,cdr3_amino_acid,HLA-A*33:01,HLA-A*33:03,HLA-A*26:01,HLA-A*31:01,HLA-A*11:01,HLA-A*23:01,HLA-A*03:01,HLA-A*24:03,HLA-A*34:01,HLA-A*03:02,HLA-A*25:01,HLA-A*32:01,HLA-A*01:01,HLA-A*29:02,HLA-A*29:01,HLA-A*66:01,HLA-A*02:01,HLA-A*24:02,HLA-A*02:05,HLA-A*02:06,HLA-A*30:01,HLA-A*30:02,HLA-A*68:01,HLA-A*68:02,HLA-B*55:01,HLA-B*38:01,HLA-B*38:02,HLA-B*15:17,HLA-B*50:01,HLA-B*45:01,HLA-B*51:01,HLA-B*57:01,HLA-B*49:01,HLA-B*14:02,HLA-B*52:01,HLA-B*48:01,HLA-B*58:01,HLA-B*40:02,HLA-B*40:01,HLA-B*40:06,HLA-B*18:01,HLA-B*27:05,HLA-B*15:03,HLA-B*39:06,HLA-B*15:01,HLA-B*13:02,HLA-B*39:01,HLA-B*15:07,HLA-B*14:01,HLA-B*44:03,HLA-B*44:02,HLA-B*08:01,HLA-B*15:18,HLA-B*07:02,HLA-B*41:01,HLA-B*41:02,HLA-B*35:08,HLA-B*07:05,HLA-B*35:03,HLA-B*35:02,HLA-B*35:01,HLA-B*56:01,HLA-B*37:01,HLA-B*53:01,HLA-C*12:02,HLA-C*12:03,HLA-C*06:02,HLA-C*05:01,HLA-C*14:02,HLA-C*04:01,HLA-C*03:04,HLA-C*03:02,HLA-C*03:03,HLA-C*01:02,HLA-C*15:05,HLA-C*15:02,HLA-C*08:03,HLA-C*08:02,HLA-C*08:01,HLA-C*16:01,HLA-C*16:02,HLA-C*02:02,HLA-C*07:02,HLA-C*17:01,HLA-C*07:01,HLA-C*07:04,HLA-DRB1*12:01,HLA-DRB1*12:02,HLA-DRB1*11:01,HLA-DRB1*11:03,HLA-DRB1*11:02,HLA-DRB1*11:04,HLA-DRB1*04:03,HLA-DRB1*04:02,HLA-DRB1*04:01,HLA-DRB1*14:01,HLA-DRB1*04:07,HLA-DRB1*14:07,HLA-DRB1*14:04,HLA-DRB1*04:04,HLA-DRB1*04:08,HLA-DRB1*09:01,HLA-DRB1*03:01,HLA-DRB1*13:03,HLA-DRB1*14:02,HLA-DRB1*01:01,HLA-DRB1*01:02,HLA-DRB1*01:03,HLA-DRB1*13:05,HLA-DRB1*15:01,HLA-DRB1*13:02,HLA-DRB1*13:01,HLA-DRB1*15:02,HLA-DRB1*08:04,HLA-DRB1*08:03,HLA-DRB1*15:03,HLA-DRB1*08:01,HLA-DRB1*16:01,HLA-DRB1*16:02,HLA-DRB1*07:01,HLA-DRB1*04:05,HLA-DRB1*10:01,HLA-DRB1*14:54,HLA-DRB1*08:02,HLA-DQA1*01:04,HLA-DQA1*01:05,HLA-DQA1*04:01,HLA-DQA1*01:01,HLA-DQA1*01:02,HLA-DQA1*01:03,HLA-DQA1*06:01,HLA-DQA1*03:02,HLA-DQA1*03:03,HLA-DQA1*03:01,HLA-DQA1*02:01,HLA-DQA1*05:01,HLA-DQA1*05:05,HLA-DQB1*04:02,HLA-DQB1*04:01,HLA-DQB1*05:03,HLA-DQB1*06:09,HLA-DQB1*06:04,HLA-DQB1*03:05,HLA-DQB1*03:02,HLA-DQB1*03:03,HLA-DQB1*06:03,HLA-DQB1*06:02,HLA-DQB1*03:01,HLA-DQB1*06:01,HLA-DQB1*02:01,HLA-DQB1*05:01,HLA-DQB1*05:02,HLA-DQB1*02:02,HLA-DPA1*02:01,HLA-DPA1*01:04,HLA-DPA1*02:02,HLA-DPA1*01:03,HLA-DPB1*04:02,HLA-DPB1*04:01,HLA-DPB1*14:01,HLA-DPB1*01:01,HLA-DPB1*17:01,HLA-DPB1*15:01,HLA-DPB1*02:02,HLA-DPB1*06:01,HLA-DPB1*13:01,HLA-DPB1*03:01,HLA-DPB1*02:01,HLA-DPB1*104:01,HLA-DPB1*16:01,HLA-DPB1*11:01,HLA-DPB1*05:01,HLA-DPB1*10:01,HLA-DPAB*02:01_04:01,HLA-DQAB*05:05_06:04,HLA-DPAB*02:01_04:02,HLA-DRDQ*10:01_01:05_05:01,HLA-DQAB*05:05_03:01,HLA-DQAB*05:05_03:02,HLA-DQAB*05:05_03:03,HLA-DQAB*03:03_05:01,HLA-DPAB*01:03_11:01,HLA-DPAB*02:01_13:01,HLA-DQAB*03:03_03:01,HLA-DQAB*03:03_03:02,HLA-DPAB*02:01_03:01,HLA-DQAB*02:01_03:01,HLA-DQAB*03:03_06:02,HLA-DQAB*02:01_03:03,HLA-DQAB*02:01_03:02,HLA-DQAB*05:01_02:02,HLA-DQAB*06:01_03:01,HLA-DPAB*01:03_17:01,HLA-DPAB*01:03_05:01,HLA-DPAB*02:01_01:01,HLA-DQAB*01:01_05:03,HLA-DQAB*01:01_05:01,HLA-DQAB*01:01_02:02,HLA-DPAB*02:02_01:01,HLA-DQAB*05:01_03:01,HLA-DQAB*01:02_02:01,HLA-DRDQ*03:01_05:01_02:01,HLA-DQAB*01:01_03:01,HLA-DQAB*01:01_03:02,HLA-DQAB*04:01_04:02,HLA-DQAB*01:04_02:01,HLA-DQAB*01:02_05:01,HLA-DQAB*01:02_05:02,HLA-DQAB*05:05_06:03,HLA-DQAB*05:05_06:02,HLA-DQAB*03:01_03:02,HLA-DPAB*02:02_02:01,HLA-DQAB*03:01_03:01,HLA-DRDQ*13:01_01:03_06:03,HLA-DQAB*01:03_02:01,HLA-DQAB*01:03_02:02,HLA-DPAB*02:02_04:02,HLA-DPAB*02:02_04:01,HLA-DQAB*01:04_05:03,HLA-DPAB*02:01_10:01,HLA-DQAB*05:05_02:02,HLA-DQAB*05:05_02:01,HLA-DQAB*02:01_02:02,HLA-DQAB*02:01_02:01,HLA-DQAB*02:01_05:01,HLA-DPAB*02:01_11:01,HLA-DQAB*03:03_02:02,HLA-DQAB*03:03_02:01,HLA-DPAB*02:01_02:01,HLA-DPAB*01:03_03:01,HLA-DRDQ*15:01_01:02_06:02,HLA-DQAB*03:01_06:03,HLA-DQAB*03:01_06:02,HLA-DQAB*03:01_06:04,HLA-DPAB*01:03_01:01,HLA-DQAB*01:02_06:03,HLA-DQAB*04:01_05:01,HLA-DPAB*01:03_13:01,HLA-DQAB*01:02_03:03,HLA-DQAB*01:02_06:09,HLA-DQAB*01:02_03:01,HLA-DQAB*05:01_03:03,HLA-DQAB*05:01_06:03,HLA-DQAB*05:01_06:02,HLA-DQAB*05:01_06:04,HLA-DQAB*03:01_02:01,HLA-DQAB*03:01_02:02,HLA-DPAB*01:03_02:01,HLA-DRDQ*09:01_03:02_03:03,HLA-DQAB*01:02_06:0

In [54]:
# Step 0: Read in everything
metadata = pd.read_csv('data/processed_files/metadata_merged_full.tsv', sep='\t')
cdr3_files = pd.read_csv('data/cdr3_files.csv')
hla_info = pd.read_csv('data/hla_info.csv')

# Step 1: Extracting file_ids for all rows - We convert the string to a list 
cdr3_files['file_id'] = cdr3_files['file_id'].str.split(',')

metadata = metadata[:10000]
cdr3_files = cdr3_files[:10000]
hla_info = hla_info[:10000]

In [44]:
file_id = "P00030"
# matching_row = metadata.loc[metadata['sample_name'] == file_id, metadata.columns[18:288]]
matching_row = metadata.loc[metadata['sample_name'] == file_id]
true_hla_columns = matching_row.columns[18:288][matching_row.iloc[0, 18:288]].tolist()

true_hla_columns

['HLA-A*02:01',
 'HLA-A*24:02',
 'HLA-B*51:01',
 'HLA-B*08:01',
 'HLA-C*14:02',
 'HLA-C*07:02',
 'HLA-DRB1*04:03',
 'HLA-DQA1*03:01',
 'HLA-DQB1*03:05',
 'HLA-DPA1*01:03',
 'HLA-DPB1*04:01',
 'HLA-DPB1*02:01',
 'HLA-DPAB*01:03_02:01',
 'HLA-DPAB*01:03_04:01']

In [51]:
from joblib import Parallel, delayed

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
metadata.fillna(False, inplace=True)



# Define a function to process each row
def process_row(row):
    file_ids = row['file_id']
    cdr3_amino_acid = row['cdr3_amino_acid']
    
    # Initialize hla_info for current cdr3_amino_acid
    if cdr3_amino_acid not in hla_info:
        hla_info[cdr3_amino_acid] = {}

    # Iterate over each file_id for the current row
    for file_id in file_ids:

        # Look for the row "file_id" in metadata
        matching_row = metadata.loc[metadata['sample_name'] == file_id]

        # Extract the HLA columns set to True
        true_hla_columns = matching_row.columns[18:288][matching_row.iloc[0, 18:288]].tolist()

        # Update hla_info dictionary with True values for the current cdr3_amino_acid
        for hla_column in true_hla_columns:
            hla_info[cdr3_amino_acid][hla_column] = True

# Use joblib to parallelize processing of rows
Parallel(n_jobs=-1)(delayed(process_row)(row) for index, row in tqdm(cdr3_files.iterrows(), total=len(cdr3_files)))


In [ ]:
hla_info = {}

# Iterate over each row in cdr3_files dataframe
for index, row in tqdm(cdr3_files.iterrows(), total=len(cdr3_files)):

    file_ids = row['file_id']
    cdr3_amino_acid = row['cdr3_amino_acid']
    
    # Initialize hla_info for current cdr3_amino_acid
    if cdr3_amino_acid not in hla_info:
        hla_info[cdr3_amino_acid] = {}

    # Iterate over each file_id for the current row
    for file_id in file_ids:

        # Look for the row "file_id" in metadata
        matching_row = metadata.loc[metadata['sample_name'] == file_id]

        # Extract the HLA columns set to True
        true_hla_columns = matching_row.columns[18:288][matching_row.iloc[0, 18:288]].tolist()

        # Update hla_info dictionary with True values for the current cdr3_amino_acid
        for hla_column in true_hla_columns:
            hla_info[cdr3_amino_acid][hla_column] = True

In [9]:

distinct_tcr = tcr_df["cdr3"]

tcr_samples = pd.DataFrame(np.nan, index=distinct_tcr, columns=list_files)

# Replace NaN values with False
tcr_samples.fillna(False, inplace=True)
tcr_samples

sample_name,P00030,P00061,P00127,P00033,P00175,P00078,P00173,P00069,P00093,P00092,P00121,P00024,P00042,P00167,P00160,P00161,P00040,P00036,P00075,P00126,P00080,P00198,P00007,P00064,P00088,P00058,P00163,P00021,P00195,P00105,P00015,P00102,P00199,P00051,P00048,P00098,P00083,P00076,P00087,P00162,P00191,P00146,P00106,P00090,P00137,P00115,P00027,P00037,P00186,P00178,P00089,P00029,P00096,P00071,P00057,P00159,P00063,P00138,P00019,P00124,P00194,P00085,P00113,P00072,P00097,P00182,P00050,P00066,P00149,P00017,P00166,P00169,P00052,P00044,P00176,P00002,P00067,P00174,P00043,P00132,P00164,P00008,P00143,P00079,P00062,P00041,P00012,P00152,P00100,P00189,P00028,P00032,P00170,P00046,P00065,P00180,P00193,P00023,P00150,P00187,P00151,P00049,P00185,P00060,P00135,P00055,P00128,P00074,P00006,P00020,P00197,P00122,P00016,P00142,P00141,P00084,P00045,P00001,P00140,P00117,P00094,P00158,P00005,P00145,P00034,P00153,P00109,P00144,P00133,P00104,P00013,P00039,P00101,P00107,P00025,P00192,P00059,P00010,P00095,P00003,P00073,P00018,P00110,P00004,P00183,P00026,P00056,P00184,P00014,P00068,P00114,P00009,P00129,P00147,P00171,P00165,P00081,P00054,P00131,P00116,P00155,P00179,P00172,P00190,P00156,P00196,P00125,P00077,P00148,P00181,P00011,P00047,P00035,P00031,P00119,P00053,P00134,P00154,P00168,P00099,P00157,P00120,P00139,P00188,P00200,P00038,P00091,P00177,P00070,P00118,P00130,P00123,P00136,P00111,P00108,P00022,P00103,P00112,P00086,P00463,P00531,P00571,P00424,P00477,P00601,P00534,P00629,P00441,P00503,P00507,P00520,P00644,P00444,P00529,P00483,P00536,P00476,P00538,P00578,P00430,P00457,P00528,P00478,P00432,P00480,P00423,P00607,P00450,P00653,P00576,P00608,P00414,P00546,P00620,P00466,P00663,P00502,P00595,P00589,P00651,P00508,P00646,P00543,P00491,P00407,P00451,P00433,P00497,P00437,P00500,P00501,P00459,P00448,P00442,P00645,P00573,P00456,P00584,P00406,P00558,P00443,P00446,P00438,P00559,P00425,P00622,P00624,P00484,P00435,P00467,P00554,P00597,P00609,P00596,P00619,P00524,P00428,P00440,P00606,P00577,P00515,P00506,P00458,P00481,P00614,P00518,P00479,P00592,P00447,P00593,P00493,P00488,P00550,P00652,P00660,P00656,P00582,P00627,P00562,P00570,P00434,P00547,P00650,P00598,P00405,P00449,P00418,P00420,P00564,P00626,P00409,P00658,P00666,P00513,P00642,P00553,P00517,P00561,P00555,P00523,P00623,P00591,P00504,P00602,P00575,P00421,P00499,P00445,P00617,P00605,P00498,P00413,P00657,P00417,P00599,P00647,P00628,P00638,P00472,P00549,P00566,P00495,P00510,P00581,P00560,P00662,P00460,P00648,P00635,P00521,P00410,P00637,P00567,P00473,P00485,P00574,P00636,P00469,P00486,P00404,P00512,P00487,P00611,P00530,P00427,P00557,P00422,P00632,P00455,P00633,P00532,P00522,P00640,P00426,P00544,P00585,P00516,P00649,P00415,P00586,P00542,P00639,P00634,P00453,P00563,P00462,P00494,P00454,P00583,P00610,P00408,P00545,P00514,P00411,P00431,P00509,P00439,P00489,P00659,P00505,P00665,P00537,P00419,P00604,P00603,P00590,P00526,P00569,P00625,P00667,P00551,P00621,P00630,P00496,P00533,P00468,P00436,P00412,P00475,P00548,P00402,P00600,P00541,P00525,P00461,P00580,P00482,P00416,P00613,P00471,P00429,P00641,P00470,P00452,P00587,P00618,P00568,P00565,P00556,P00668,P00465,P00631,P00615,P00552,P00654,P00403,P00588,P00474,P00535,P00612,P00492,P00464,P00616,P00540,P00661,P00519,P00594,P00527,P00511,P00579,P00655,P00539,P00490,P00643,P00572,P00247,P00204,P00292,P00214,P00237,P00205,P00213,P00211,P00309,P00297,P00314,P00256,P00261,P00343,P00293,P00388,P00339,P00236,P00316,P00368,P00266,P00332,P00253,P00322,P00337,P00307,P00350,P00363,P00289,P00229,P00249,P00251,P00202,P00206,P00366,P00383,P00260,P00268,P00216,P00243,P00276,P00360,P00281,P00394,P00385,P00282,P00386,P00370,P00219,P00242,P00274,P00348,P00280,P00299,P00387,P00250,P00396,P00358,P00278,P00331,P00306,P00374,P00269,P00382,P00398,P00279,P00326,P00255,P00390,P00265,P00338,P00308,P00290,P00330,P00215,P00244,P00303,P00212,P00241,P00311,P00312,P00377,P00354,P00259,P00327,P00392,P00310,P00233,P00342,P00344,P00232,P00273,P00230,P00283,P00353,P00222,P00272,P00228,P00334,P00323,P00347,P00317,P00245,P00225,P0032

In [6]:
# Create an empty dataframe
df = pd.DataFrame(columns=list_files)
distinct_tcr = tcr_df["cdr3"]

# Add rows with empty values
df = df.append([[] for _ in range(len(distinct_tcr))], ignore_index=True)

# Set index to tcr_list values
df.set_index(distinct_tcr, inplace=True)

print(df)

                P00030 P00061 P00127 P00033 P00175 P00078 P00173 P00069  \
cdr3                                                                      
CASSLEGTGNTEAFF    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CASSLGETQYF        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CASSLPGQDYEQYF     NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CASSLGYEQYF        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CASSYRGPSYEQYF     NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...                ...    ...    ...    ...    ...    ...    ...    ...   
CATSRDLLDTQYF      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CSAPQGYTEAFF       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CASSQALAGVGEQFF    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CASSQADREKLFF      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
CAWSSRGEQFF        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                P00093 P